## Hands-on session 1.2 - advection model continued: refining with built-in models 

This notebook continues from session 1.1, moving away from building all terms by hand and utilizing the common_models module to refactor the existing code. The concept of adding diagnostic manipulators for individual variables to the run is also covered here.

The reader is left with the exercise of adding an outflow boundary condition using one of the common models, as well as adding the advection of momentum in order to produce a shock. 

**NOTE**: The default gradient and divergence operators on a staggered grid are central and second-order accurate, and will produce produce grid-scale oscillations in the presence of shocks.

Demonstrated concepts:

- Common models for advection and pressure gradients 
- Setting outflow boundary condition on advection models
- Querying the wrapper on added models and terms
- Diagnostic variables and manipulators
- Using the built-in dashboard to visualise fluid variable evolution

In [28]:
from RMK_support import RKWrapper ,Grid
import RMK_support.simple_containers as sc
import RMK_support.common_models as cm
import RMK_support.IO_support as io

import numpy as np
import holoviews as hv
import matplotlib.pyplot as plt

### Pre-model initialization

This follows as in session 1.1

In [29]:
#Wrapper initialization
rk = RKWrapper()

#I/O setuo
rk.jsonFilepath = "./config.json" 
hdf5Filepath = "./RMKOutput/day_1_2/"
rk.setHDF5Path(hdf5Filepath) 

#MPI setup
rk.setMPIData(numProcsX=4)

# Normalization
rk.setNormDensity(1.0e19) 
rk.setNormTemperature(10.0) 
rk.setNormRefZ(1.0) 

#Grid initialization
xGridWidths = 0.025*np.ones(512)
gridObj = Grid(xGridWidths, interpretXGridAsWidths=True)
rk.grid = gridObj

#Variables

n = 1 + np.exp(-(gridObj.xGrid-np.mean(gridObj.xGrid))**2) 
T = np.ones(len(gridObj.xGrid)) 

rk.addVarAndDual('n',n,isCommunicated=True) 
rk.addVarAndDual('T',T,isDerived=True) 
rk.addVarAndDual('G',isCommunicated=True,primaryOnDualGrid=True) 
rk.addVarAndDual('u',isDerived=True,derivationRule=sc.derivationRule("flowSpeedFromFlux",["G_dual","n_dual"]),primaryOnDualGrid=True,isCommunicated=True)
rk.addVar('time',isDerived=True,isScalar=True)

### Models and Terms

The module `common_models` is supplied to speed up the initialization of models and terms that show up often. 

**NOTE**: Many models in `common_models` assume the default normalization as available from `sk_normalization`.

The advection model for the density variable can be added using the `common_models` function `staggeredAdvection`:

In [30]:
cm.staggeredAdvection?

Signature:
cm.staggeredAdvection(
    modelTag: str,
    advectedVar: str,
    fluxVar: str,
    advectionSpeed: Union[str, NoneType] = None,
    staggeredAdvectionSpeed: Union[str, NoneType] = None,
    lowerBoundVar: Union[str, NoneType] = None,
    leftOutflow=False,
    rightOutflow=False,
    staggeredAdvectedVar=False,
    vData=<RMK_support.simple_containers.VarData object at 0x7fba60f9f340>,
) -> RMK_support.simple_containers.CustomModel
Docstring:
Create a staggered grid advection model

Args:
    modelTag (str): Model tag
    fluxVar (str): Name of flux variable - always on dual grid
    advectionSpeed (Union[str,None], optional): Name of advection speed variable - on regular grid - only required if there is outflow. Defaults to None.
    staggeredAdvectionSpeed (Union[str,None], optional): Name of advection speed variable - on dual grid - required if staggeredAdvectedVar=True. Defaults to None.
    lowerBoundVar (Union[str,None], optional): Name of outflow lower bound variab

Consulting the above documentation, and remembering that the density is advected by the flow speed 'u', add an outflow boundary condition on the right side of the domain.

In [31]:
# NOTE: There are issues in the docstring - it's missing bits and in wrong order!!
# Outflow always needs a non-staggered advection speed due to the BC implementation.
# Otherwise if advecting staggered variables you need to to specify staggered advection speed.
rk.addModel(cm.staggeredAdvection("nAdvection","n","G_dual", advectionSpeed = "u", rightOutflow = True)) # <- YOUR CODE HERE

Checking terms in model nAdvection:
   Checking term divFlux
   Checking term rightBC


Similarly, the pressure gradient can be added in a one-liner using the `staggeredPressureGrad` function


Here we need to supply a temperature variable (hence its inclusion in session 1.1!) and species mass (this time in SI!)

In [32]:
massRatio = 1/1836

rk.addModel(cm.staggeredPressureGrad('pGrad',"G_dual","n","T",9.1093837e-31/massRatio))

Checking terms in model pGrad:
   Checking term bulkGrad


The momentum equation with momentum advection is given as 

$$m_i \frac{\partial \Gamma}{\partial t} = - m_i\frac{\partial (\Gamma u)}{\partial x} - \frac{\partial (nkT)}{\partial x}$$

Look at the documentation for `staggeredAdvection`, remembering that the flux lives on the dual grid and that the velocity normalization obeys $u_0=x_0/t_0$. Use it to add another model that would implement the first RHS term in the above equation. Remember that an ouflow boundary condition is required here too!


In [33]:
# Advecting flux with a mass multiplier which I assume is in the normalisation?
# Using implicit variable

In [34]:
# The flux is the flux of flux, which has to be constructed by multiplying G by velocity, which is why
# you use the staggeredAdvectedVar = True setting.
# You are also advecting G which is on the staggered grid so advect G_dual.

rk.addModel(cm.staggeredAdvection("GAdvection", "G_dual", "", staggeredAdvectedVar = True, advectionSpeed = "u", staggeredAdvectionSpeed = "u_dual", rightOutflow = True))

Checking terms in model GAdvection:
   Checking term divFlux
   Checking term rightBC


### Time integration options

These are set in the same way as in session 1.1


In [35]:
rk.addIntegrator("BE",sc.picardBDEIntegrator(nonlinTol=1e-12,absTol=10.0,convergenceVars=['n','G_dual']) )

rk.setIntegratorGlobalData(initialTimestep=0.1) 

bdeStep = sc.IntegrationStep("BE")

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("StepBDE",bdeStep.dict())

rk.setFixedNumTimesteps(10000)
rk.setFixedStepOutput(200)

### Query functions on the wrapper

The following functions are useful when inspecting already constructed wrappers:

In [36]:
rk.varList()

['n', 'n_dual', 'T', 'T_dual', 'G_dual', 'G', 'u_dual', 'u', 'time']

In [37]:
rk.modelTags()

['nAdvection', 'pGrad', 'GAdvection']

In [38]:
rk.getTermsThatEvolveVar('n')

[('nAdvection', 'divFlux'), ('nAdvection', 'rightBC')]

For more information on the wrapper see the [rk_wrapper](https://remkit1d-python.readthedocs.io/en/latest/RMK_support.html#module-RMK_support.rk_wrapper) documentation.

### Adding diagnostic variables

In many cases we would like to inspect individual terms in various equations. One way of doing this in ReMKiT1D for custom models is to attach a set of manipulators to a variable name. This is all abstracted in the wrapper function `addTermDiagnosisForVars`

Try adding diagnosis for the other evolved variable representing the flux $\Gamma$.

In [39]:
rk.addTermDiagnosisForVars(['n'])

This has now added two new variables to evaluate terms that evolve the density and we can check this

In [40]:
rk.varList()

['n',
 'n_dual',
 'T',
 'T_dual',
 'G_dual',
 'G',
 'u_dual',
 'u',
 'time',
 'nAdvectiondivFlux',
 'nAdvectionrightBC']

### Create config 

In [41]:
rk.writeConfigFile()

### Set global plotting options

In [42]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=150,dpi=150)

<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAYAAACqaXHeAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz
AAAB+wAAAfsBxc2miwAAABl0RVh0U29mdHdhcmUAd3d3Lmlua3NjYXBlLm9yZ5vuPBoAAA6zSURB
VHic7ZtpeFRVmsf/5966taWqUlUJ2UioBBJiIBAwCZtog9IOgjqACsogKtqirT2ttt069nQ/zDzt
tI4+CrJIREFaFgWhBXpUNhHZQoKBkIUASchWla1S+3ar7r1nPkDaCAnZKoQP/D7mnPOe9/xy76n3
nFSAW9ziFoPFNED2LLK5wcyBDObkb8ZkxuaoSYlI6ZcOKq1eWFdedqNzGHQBk9RMEwFAASkk0Xw3
ETacDNi2vtvc7L0ROdw0AjoSotQVkKSvHQz/wRO1lScGModBFbDMaNRN1A4tUBCS3lk7BWhQkgpD
lG4852/+7DWr1R3uHAZVQDsbh6ZPN7CyxUrCzJMRouusj0ipRwD2uKm0Zn5d2dFwzX1TCGhnmdGo
G62Nna+isiUqhkzuKrkQaJlPEv5mFl2fvGg2t/VnzkEV8F5ioioOEWkLG86fvbpthynjdhXYZziQ
x1hC9J2NFyi8vCTt91Fh04KGip0AaG9zuCk2wQCVyoNU3Hjezee9bq92duzzTmxsRJoy+jEZZZYo
GTKJ6SJngdJqAfRzpze0+jHreUtPc7gpBLQnIYK6BYp/uGhw9YK688eu7v95ysgshcg9qSLMo3JC
4jqLKQFBgdKDPoQ+Pltb8dUyQLpeDjeVgI6EgLIQFT5tEl3rn2losHVsexbZ3EyT9wE1uGdkIPcy
BGxn8QUq1QrA5nqW5i2tLqvrrM9NK6AdkVIvL9E9bZL/oyfMVd/jqvc8LylzRBKDJSzIExwhQzuL
QYGQj4rHfFTc8mUdu3E7yoLtbTe9gI4EqVgVkug2i5+uXGo919ixbRog+3fTbQ8qJe4ZOYNfMoTI
OoshUNosgO60AisX15aeI2PSIp5KiFLI9ubb1vV3Qb2ltwLakUCDAkWX7/nHKRmmGIl9VgYsUhJm
2NXjKYADtM1ygne9QQDIXlk49FBstMKx66D1v4+XuQr7vqTe0VcBHQlRWiOCbmmSYe2SqtL6q5rJ
zsTb7lKx3FKOYC4DoqyS/B5bvLPxvD9Qtf6saxYLQGJErmDOdOMr/zo96km1nElr8bmPOBwI9COv
HnFPRIwmkSOv9kcAS4heRsidOkpeWBgZM+UBrTFAXNYL5Vf2ii9c1trNzpYdaoVil3WIc+wdk+gQ
noie3ecCcxt9ITcLAPWt/laGEO/9U6PmzZkenTtsSMQ8uYywJVW+grCstAvCIaAdArAsIWkRDDs/
KzLm2YcjY1Lv0UdW73HabE9n6V66cxSzfEmuJssTpKGVp+0vHq73FwL46eOjpMpbRAnNmJFrGJNu
Ukf9Yrz+3rghiumCKNXXWPhLYcjxGsIpoCMsIRoFITkW8AuyM8jC1+/QLx4bozCEJIq38+1rtpR6
V/yzb8eBlRb3fo5l783N0CWolAzJHaVNzkrTzlEp2bQ2q3TC5gn6wpnoQAmwSiGh2GitnTmVMc5O
UyfKWUKCIsU7+fZDKwqdT6DDpvkzAX4/+AMFjk0tDp5GRXLpQ2MUmhgDp5gxQT8+Y7hyPsMi8uxF
71H0oebujHALECjFKaW9Lm68n18wXp2kVzIcABytD5iXFzg+WVXkegpAsOOYziqo0OkK76GyquC3
ltZAzMhhqlSNmmWTE5T6e3IN05ITFLM4GdN0vtZ3ob8Jh1NAKXFbm5PtLU/eqTSlGjkNAJjdgn/N
aedXa0tdi7+t9G0FIF49rtMSEgAs1kDLkTPO7ebm4IUWeyh1bKomXqlgMG6kJmHcSM0clYLJ8XtR
1GTnbV3F6I5wCGikAb402npp1h1s7LQUZZSMIfALFOuL3UUrfnS8+rez7v9qcold5tilgHbO1fjK
9ubb17u9oshxzMiUBKXWqJNxd+fqb0tLVs4lILFnK71H0Ind7uiPgACVcFJlrb0tV6DzxqqTIhUM
CwDf1/rrVhTa33/3pGPxJYdQ2l2cbgVcQSosdx8uqnDtbGjh9SlDVSMNWhlnilfqZk42Th2ZpLpf
xrHec5e815zrr0dfBZSwzkZfqsv+1FS1KUknUwPARVvItfKUY+cn57yP7qv07UE3p8B2uhUwLk09
e0SCOrK+hbdYHYLjRIl71wWzv9jpEoeOHhGRrJAzyEyNiJuUqX0g2sBN5kGK6y2Blp5M3lsB9Qh4
y2Ja6x6+i0ucmKgwMATwhSjdUu49tKrQ/pvN5d53ml2CGwCmJipmKjgmyuaXzNeL2a0AkQ01Th5j
2DktO3Jyk8f9vcOBQHV94OK+fPumJmvQHxJoWkaKWq9Vs+yUsbq0zGT1I4RgeH2b5wef7+c7bl8F
eKgoHVVZa8ZPEORzR6sT1BzDUAD/d9F78e2Tzv99v8D+fLVTqAKAsbGamKey1Mt9Ann4eH3gTXTz
idWtAJ8PQWOk7NzSeQn/OTHDuEikVF1R4z8BQCy+6D1aWRfY0tTGG2OM8rRoPaeIj5ZHzJxszElN
VM8K8JS5WOfv8mzRnQAKoEhmt8gyPM4lU9SmBK1MCQBnW4KONT86v1hZ1PbwSXPw4JWussVjtH9Y
NCoiL9UoH/6PSu8jFrfY2t36erQHXLIEakMi1SydmzB31h3GGXFDFNPaK8Rme9B79Ixrd0WN+1ij
NRQ/doRmuFLBkHSTOm5GruG+pFjFdAmorG4IXH1Qua6ASniclfFtDYt+oUjKipPrCQB7QBQ2lrgP
fFzm+9XWUtcqJ3/5vDLDpJ79XHZk3u8nGZ42qlj1+ydtbxysCezrydp6ugmipNJ7WBPB5tydY0jP
HaVNzs3QzeE4ZpTbI+ZbnSFPbVOw9vsfnVvqWnirPyCNGD08IlqtYkh2hjZ5dErEQzoNm+6ykyOt
Lt5/PQEuSRRKo22VkydK+vvS1XEKlhCJAnsqvcVvH7f/ZU2R67eXbMEGAMiIV5oWZWiWvz5Fv2xG
sjqNJQRvn3Rs2lji/lNP19VjAQDgD7FHhujZB9OGqYxRkZxixgRDVlqS6uEOFaJUVu0rPFzctrnF
JqijImVp8dEKVWyUXDk92zAuMZ6bFwpBU1HrOw6AdhQgUooChb0+ItMbWJitSo5Ws3IAOGEOtL53
0vHZih9sC4vtofZ7Qu6523V/fmGcds1TY3V36pUsBwAbSlxnVh2xLfAD/IAIMDf7XYIkNmXfpp2l
18rkAJAy9HKFaIr/qULkeQQKy9zf1JgDB2uaeFNGijo5QsUyacNUUTOnGO42xSnv4oOwpDi1zYkc
efUc3I5Gk6PhyTuVKaOGyLUAYPGIoY9Pu/atL/L92+4q9wbflRJ2Trpm/jPjdBtfnqB/dIThcl8A
KG7hbRuKnb8qsQsVvVlTrwQAQMUlf3kwJI24Z4JhPMtcfng5GcH49GsrxJpGvvHIaeem2ma+KSjQ
lIwUdYyCY8j4dE1KzijNnIP2llF2wcXNnsoapw9XxsgYAl6k+KzUXbi2yP3KR2ecf6z3BFsBICdW
nvnIaG3eHybqX7vbpEqUMT+9OL4Qpe8VON7dXuFd39v19FoAABRVePbGGuXTszO0P7tu6lghUonE
llRdrhArLvmKdh9u29jcFiRRkfLUxBiFNiqSU9icoZQHo5mYBI1MBgBH6wMNb+U7Pnw337H4gi1Y
ciWs+uks3Z9fztUvfzxTm9Ne8XXkvQLHNytOOZeiD4e0PgkAIAYCYknKUNUDSXEKzdWNpnil7r4p
xqkjTarZMtk/K8TQ6Qve78qqvXurGwIJqcOUKfUWHsm8KGvxSP68YudXq4pcj39X49uOK2X142O0
Tz5/u/7TVybqH0rSya6ZBwD21/gubbrgWdDgEOx9W

### Load data from ReMKiT1D output files

In [43]:
numFiles = 50
loadpath = hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]
loadedData = io.loadFromHDF5(rk.varCont,filepaths=loadFilenames)
loadedData

<xarray.Dataset>
Dimensions:            (x: 512, h: 1, v: 1, time: 51)
Coordinates:
  * x                  (x) float64 0.0125 0.0375 0.0625 ... 12.74 12.76 12.79
  * h                  (h) int64 0
  * v                  (v) float64 1.0
  * time               (time) float64 0.0 20.0 40.0 60.0 ... 960.0 980.0 1e+03
Data variables:
    n                  (time, x) float64 1.0 1.0 1.0 ... 0.5206 0.5207 0.5209
    n_dual             (time, x) float64 1.0 1.0 1.0 ... 0.5207 0.5208 0.5209
    T                  (time, x) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
    T_dual             (time, x) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
    G_dual             (time, x) float64 0.0 0.0 0.0 ... 0.008218 0.008232 0.0
    G                  (time, x) float64 0.0 0.0 0.0 ... 0.008225 0.008239
    u_dual             (time, x) float64 0.0 0.0 0.0 0.0 ... 0.01578 0.01581 0.0
    u                  (time, x) float64 0.0 0.0 0.0 ... 0.01577 0.01579 0.01582
    nAdvectiondivFlux  (time, x) float64 0.0 0.0 0.0 ... -0.0005667 0.3293
    nAdvectionrightBC  (time, x) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 -0.3298

### Explore data with built-in dashboard

`RMK_support` offers a simple built-in dashboard to quickly explore variables 

In [44]:
import panel as pn 
import RMK_support.dashboard_support as ds

pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData,rk.grid)

dashboard.fluid2Comparison().show() # Removing .show() should display the dashboard inline - this can be buggy in some situations


Launching server at http://localhost:38583


/usr/local/lib/python3.8/dist-packages/holoviews/core/spaces.py:497: ParamFutureWarning: Parameter 'callable' on <class 'holoviews.core.spaces.Callable'> is being given a valid parameter reference <function _param_bind.<locals>.wrapped at 0x7fba9d9460d0> but is implicitly allow_refs=False. In future allow_refs will be enabled by default and the reference <function _param_bind.<locals>.wrapped at 0x7fba9d9460d0> will be resolved to its underlying value :Curve   [time]   (n). Please explicitly set allow_ref on the Parameter definition to declare whether references should be resolved or not.
  super().__init__(callable=callable,
/usr/local/lib/python3.8/dist-packages/holoviews/core/spaces.py:497: ParamFutureWarning: Parameter 'callable' on <class 'holoviews.core.spaces.Callable'> is being given a valid parameter reference <function _param_bind.<locals>.wrapped at 0x7fba9d910dc0> but is implicitly allow_refs=False. In future allow_refs will be enabled by default and the reference <function